In [1]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [2]:
import numpy as np
import time
from datetime import timedelta, datetime

In [3]:
def calculate(a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)
    
    radians=np.arctan2(c[1]-b[1],c[0]-b[0])- np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(radians*180.0/np.pi)
    
    if angle>180.0:
        angle=360-angle
    return angle

def mse(img1, img2):
    h, w, r = img1.shape
    diff = cv2.subtract(img1, img2)
    err = np.sum(diff**2)
    mse = err/(float(h*w))
    return mse

def check(original,duplicate):
    difference = cv2.subtract(original, duplicate)
    b, g, r = cv2.split(difference)

    if cv2.countNonZero(b) == 0 and cv2.countNonZero(g) == 0 and cv2.countNonZero(r) == 0:
        return True
    else:
        return False


In [8]:
# For webcam input:
cap = cv2.VideoCapture(r"C:\Users\91893\machine learning\Pose Tracker\KneeBendVideo.mp4")
stage=None
TIMER=8
i=-1
j=-1
k=-1
counter=0

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
   
size = (frame_width, frame_height)
result = cv2.VideoWriter('output.avi', 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size) 

#Lets take our confidence as 0.5
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    previmg=None
    currimg=None
    
    if cap.isOpened():
        success, image = cap.read()
        previmg=image
        
    while cap.isOpened():
        success, image = cap.read()
        currimg=image
        
        if mse(previmg,currimg)==0:
            continue
        previmg=currimg
        
        if not success:
            print("Ignoring empty camera frame.")
            break
        
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
        #Make detection
        results = pose.process(image)

        # Draw the pose annotation on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #Counter
        cv2.rectangle(image, (0, 0), (255, 75), (25, 117, 16), -1)
        cv2.rectangle(image, (700, 0), (850, 75), (25, 117, 16), -1)
        
        #Extract Landmarks
        try:
            landmarks=results.pose_landmarks.landmark
            
            #Get cooardinates
            hip=[landmarks[23].x,landmarks[23].y]
            knee=[landmarks[25].x,landmarks[25].y]
            ankle=[landmarks[27].x,landmarks[27].y]
            
            #Calculate angle
            angle=calculate(hip,knee,ankle)
            
            cv2.putText(image, 'Knee Status', (65, 17),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(stage), (35, 60),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 2, cv2.LINE_AA)
        
            cv2.putText(image, 'TIMER', (400, 35),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(TIMER), (500, 45),cv2.FONT_HERSHEY_SIMPLEX, 2, (5, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(image, 'REPS', (750, 17),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), (750, 60),cv2.FONT_HERSHEY_SIMPLEX, 1.3, (5, 255, 255), 2, cv2.LINE_AA)
                   
            if angle>140 and stage=='bent' and TIMER>0:
                #print("Keep your knee bent",TIMER)
                cv2.putText(image, str("Keep your knee bent"), (270, 95),cv2.FONT_HERSHEY_SIMPLEX, 1, (40, 25, 255), 1, cv2.LINE_AA)
                cur = time.time()
                if cur-prev >= 3:
                    stage='straight'
                    i=0

            elif angle>140 and TIMER>0:
                TIMER=8
                stage='straight'

            if i>=0 and i<15:
                i=i+1
                cv2.putText(image, str("You have kept your knee straight for a long time. Start again!!"), (100, 100),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (85, 255, 25), 2, cv2.LINE_AA)

            if angle<140 and stage=='straight':
                k=0
                prev = time.time()
                stage='bent'

            if k>=0 and k<10:
                k=k+1
                cv2.putText(image, str("Timer STARTS"), (350,90),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (5, 25, 135), 2, cv2.LINE_AA)

            if angle<140 and stage=='bent' and TIMER>0:                
            #print(TIMER)
                cur = time.time()
                if cur-prev >= 1:
                    prev = cur
                    TIMER = TIMER-1

            if TIMER==0:
                if angle<140:
                    cv2.putText(image, str("Straighten your Leg to start Next REP"), (350,90),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (5, 25, 135), 2, cv2.LINE_AA)    
                if angle>140:
                    if stage=='bent':
                        j=0
                        stage='straight'
                    elif stage=='straight':
                        TIMER=8                    
                        counter+=1

            if j>=0 and j<10:
                j=j+1
                cv2.putText(image, str("You have completed successfully!!"), (150, 100),cv2.FONT_HERSHEY_SIMPLEX, 1, (75, 255, 25), 2, cv2.LINE_AA)
     
        except:
            pass
           
        #Render detection
        mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
        mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))
    
        cv2.imshow('Pose Now',image)
        result.write(image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

Ignoring empty camera frame.
